In [1]:
!nvidia-smi

Mon Mar 20 14:25:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 97%   84C    P2   222W / 370W |  21323MiB / 24265MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
|100%   

In [2]:
from models import *
from evaluation import *
from load_data import *
from training_function import *
import IPython.display
import numpy as np
import tensorflow as tf
import warnings
import os

print(tf.__version__)
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

2.5.0


In [3]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
aug_method = '_proposed'
dataset = 'mimic'
task = 'disease'

X_train, y_train, demo_train = get_data(aug_method=aug_method, dataset=dataset, data_split='train', task=task, return_demo=True)
X_val, y_val, demo_val = get_data(aug_method=aug_method, dataset=dataset, data_split='val', task=task, return_demo=True)
X_test, y_test, demo = get_data(aug_method=aug_method, dataset=dataset, data_split='test', task=task, return_demo=True)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, demo_train))
train_dataset = train_dataset.shuffle(buffer_size=len(y_train)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val, demo_val))
val_dataset = val_dataset.batch(BATCH_SIZE)

In [6]:
algo = 'ERM'
archi = 'densenet'
mode = ''
    
model_name = 'model_{archi}_{dataset}_{algo}_proposed_no_weight'.format(archi=archi, dataset=dataset, algo=algo,
                                                                                    task=task)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model = get_model(algo, 'ERM', task, archi, model_name)

# model_name = 'model_{archi}_{dataset}_{algo}_task_transfer_{task}_proposed'.format(archi=archi, dataset=dataset, algo=algo,
#                                                                                     task=task)    
checkpoint_filepath = 'checkpoints/'+model_name

checkpoint = tf.train.Checkpoint(model)
manager = tf.train.CheckpointManager(checkpoint, directory=checkpoint_filepath, max_to_keep=1, checkpoint_name=model_name)

train_step(model, algo, mode, checkpoint_filepath, optimizer, manager, train_dataset, val_dataset)

save_thresh(model, model_name, manager, X_val, y_val)

compute_original_predictions(model, model_name, manager, X_test)
    
compute_aug_predictions(model, model_name, manager)
# compute_aug_predictions_chexpert(model, model_name, manager, X_test)


In [7]:
aug_method = ''
dataset = 'mimic'
task = 'disease'

X_train, y_train, demo_train = get_data(aug_method=aug_method, dataset=dataset, data_split='train', task=task, return_demo=True)
X_val, y_val, demo_val = get_data(aug_method=aug_method, dataset=dataset, data_split='val', task=task, return_demo=True)
X_test, y_test, demo = get_data(aug_method=aug_method, dataset=dataset, data_split='test', task=task, return_demo=True)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, demo_train))
train_dataset = train_dataset.shuffle(buffer_size=len(y_train)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val, demo_val))
val_dataset = val_dataset.batch(BATCH_SIZE)

In [8]:
algo = 'ERM'
archi = 'densenet'
mode = ''
    
model_name = 'model_{archi}_{dataset}_{algo}_no_weight'.format(archi=archi, dataset=dataset, algo=algo,
                                                                                    task=task)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model = get_model(algo, 'ERM', task, archi, model_name)

# model_name = 'model_{archi}_{dataset}_{algo}_task_transfer_{task}'.format(archi=archi, dataset=dataset, algo=algo,
#                                                                                     task=task)    
checkpoint_filepath = 'checkpoints/'+model_name

checkpoint = tf.train.Checkpoint(model)
manager = tf.train.CheckpointManager(checkpoint, directory=checkpoint_filepath, max_to_keep=1, checkpoint_name=model_name)

train_step(model, algo, mode, checkpoint_filepath, optimizer, manager, train_dataset, val_dataset)

save_thresh(model, model_name, manager, X_val, y_val)

compute_original_predictions(model, model_name, manager, X_test)
    
compute_aug_predictions(model, model_name, manager)
# compute_aug_predictions_chexpert(model, model_name, manager, X_test)
